<img src='../images/gdd-logo.png' width='250px' align='right' style="padding: 15px">

# Feature Selection

So far we have considered machine learning applications with relatively few features. In practice, there might be hundreds or more columns of information that we may or may not want to use for our ML applications. But how should we make this tricky decision, and how does it actually affect our models? While ML models handle this type of data better? This and more will be the focus of this notebook.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Food Recipes dataset

The dataset that we will consider here contains over 15,000 food recipes and 445 columns of information describing it. 

We will use it to try to **predict the calorie content** of each meal using the available information. But what is this information really?

In [ ]:
food_df = pd.read_csv("../data/food_recipes.csv")
food_df.head()

There are some *continuous* variables such as protein, fat and sodium content or ingredient count. 

However, the vast majority of the available features are *binary*, indicating whether a certain ingredient or 'theme' is present for each recipe. 

445 columns is quite a lot of information to deal with! Think about the following:

- Which of these features do we actually need?
- Why could excluding some features be a good idea?
- How can we tell if a feature is/is not important?
- How may our answers alter future data collection?

Let's quickly check how many non-dummy variables we actually have among the available features:

In [ ]:
(
    food_df
    .apply(lambda col: set(col) != {0,1})
    .loc[lambda x: x == True]
    .index
)

It appears that actually only a few of the features are not indicator variables - this is something to keep in mind for our further ML application building.

Finally, before we start, let's also make sure we have sufficient variation in calories content across the dataset:

In [ ]:
sns.violinplot(x="calories", data = food_df);

<!-- <mark>**Question:** is there enough variation? -->

# 1. Preliminary feature selection

We can almost always simplify the problem at hand by separating the most valuable pieces of information from the least informative. These parts of the data may in fact do nothing but introduce noise and steal degrees of freedom from our model.

In this section we shall focus on removing features that have:
- lots of missing values,
- little variation,
- a high correlation with other features or,
- a low correlation with the target.

### 1.1 Missing value ratio
Two of the most common cases of low quality data are variables with many (or even majority) of values missing or variables with little variation. Let's have a closer look at those: 

In [ ]:
#missing values:
food_df.isna().sum().sum()

There are only 83 values missing in total, so it does not look like any variables are mostly empty. 

For simplicity we will assume that these 83 missing values are random and can be safely removed from this relatively large dataset, though in practice we may have wanted to further inspect or impute those.

### 1.2 Low variance

How do we find variables with little variation? 

In the case of continuous variables it would mean a very small spread and standard deviation; while in the case of dummy variables this would be the case when the vast majority of entries for a column is all `1` or all `0`. 

Let's try to find those! We'll start with the continuous features.

In [ ]:
fig, axs = plt.subplots(2,2, figsize = (16,10))

sns.boxplot(data = food_df, x = "protein", ax = axs[0][0])
sns.boxplot(data = food_df, x = "fat", ax = axs[0][1])
sns.boxplot(data = food_df, x = "sodium", ax = axs[1][0])
sns.boxplot(data = food_df, x = "ingr_count", ax = axs[1][1]);

It may appear that whilst the other variables have substantial variation in them, `sodium` does not. 

However, upon closer inspection, we can see that leaving extreme outliers aside, there is actually noticeable variation for the core of the data!

In [ ]:
sodium_filter = food_df.loc[lambda df: df['sodium'] <2000, 'sodium']
sns.boxplot(data = sodium_filter, orient='h');

Treating outliers is something that is beyond our scope in this notebook, so instead we will further focus on finding low-informativeness variables among the dummy features.

Let's try to identify if we have any dummy features where the vast majority of entries is either all `1` or all `0`:

In [ ]:
food_df.columns

In [ ]:
(
    food_df
    .drop(columns=['title', 'rating', 'calories', 
                   'protein', 'fat', 'sodium', 
                   'ingr_count'])#only focusing on dummy variables which excludes a few variables 
    .apply(lambda col: (col.mean() < 0.05) | (col.mean() > 0.95))
    .sum()
)

383 of the dummy features actually contain 95% or more identical entries!

for example, 
$$(0 \times 95 + 5 \times 1)/100 = 0.5$$

This means that they would not provide much information for our model. 

Whilst with some ML models we may consider keeping them anyways, most models would not get hurt much or would even benefit from exclusion of such information. We will thereby exclude these 383 columns.

*Note:* the threshold of 95% is rather arbitrary, so in the future we could build a custom transformer and tune this number as a hyper-parameter 

In [ ]:
cols_to_exclude = (
    food_df
    .drop(columns=['title', 'rating', 'calories', 
                   'protein', 'fat', 'sodium', 
                   'ingr_count'])
    .apply(lambda col: (col.mean() < 0.05) | (col.mean() > 0.95))
    .loc[lambda x: x == True]
    .index
    .tolist()
)

food_df_prepared = (
    food_df.dropna()
    .drop(cols_to_exclude, axis = 'columns')
    .reset_index(drop = True)
)

print(f'Features before: {len(food_df.columns)}')
print(f'Features after: {len(food_df_prepared.columns)}')
print(f'Features eliminated: {len(cols_to_exclude)}')

In practice we could have also used a tool from *scikit-learn* called [VarianceThreshold](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html#sklearn.feature_selection.VarianceThreshold) that would allow us to essentially do the same automatically by specifying a variance threshold. Features with variance below this threshold will be eliminated.

### <mark> Exercise
    
Implement the `VarianceThreshold` transformer from Scikit-Learn to remove redundant binary columns.

In [ ]:
# add your code here

In [ ]:
# %load ../answers/feature_selection/variance_threshold.py

### 1.3 High correlation

The feature selection above only focused on features' overall informativeness, but not how informative they could be to predict the target variable. Some variables may be completely unrelated to the target or even mainly consist of noise. 

In addition to this, some features may also be "too related" to each other and essentially be copies of the same information. We therefore may want to inspect feature 'relatedness', and a great tool for that is a **correlation map**:

In [ ]:
fig, ax = plt.subplots(figsize = (16,12))
sns.heatmap(food_df_prepared.iloc[:, 1:].corr(), cmap = sns.diverging_palette(250, 10, as_cmap=True), ax = ax);

For large sets of features like our, correlation maps may be harder to read.

However, it is still a great source of information. We are essentially looking for two things:

- Features (almost) unrelated to the target (meaning a correlation coefficient close to zero)
- Features that are very strongly correlated to each other (close to 1 or -1)

Firstly, in our case only a few variables seem to be rather strongly correlated to the target (calories content): *'protein', 'fat', 'chicken', 'dinner'* and perhaps *'vegan'* and *'vegetarian'*. In practice we could have considered only focusing on these features and see how models' performance is altered. Here we will nonetheless start with keeping all 'unrelated' features and see how we can take care of them by other means.

Secondly, we see a few features closely related to each other: 'gourmet' and 'bon_appetit' as well as 'pescatarian' and 'kosher'. We can inspect those more closely using `pd.crosstab` function:

In [ ]:
display(food_df_prepared[['gourmet','bon_appétit']].corr())
pd.crosstab(food_df_prepared['gourmet'], food_df_prepared['bon_appétit'])

In [ ]:
display(food_df_prepared[['pescatarian','kosher']].corr())
pd.crosstab(food_df_prepared['pescatarian'], food_df_prepared['kosher'])

While in each pair the two features are indeed closely related to each other, they clearly are not duplicates. We would only be concerned by a correlation very close to 1 or -1. This means that all these features can stay.

**Correlation map summary:** a great tool to only select features reasonably correlated with the target or exclude near-duplicate features.

**NB:** in practice there is a range of statistical tools to determine whether a correlation between two variables is significantly different from zero. These can be used as a tool to determine which variables to keep. However some features may appear unrelated to the target, but still provide valuable information in combination with other 'unrelated' features: **correlation measures the strength of linear relationships!** Such correlation-based exclusion decisions have to therefore be made very cautiously and conservatively.

#### Conclusion: 
* Missing value ratio: no features dropped.
* Low variance: 383 features dropped.
* High correlation: no features dropped.

Now, after we performed some key preliminary feature selection, we can use the rest with a ML model and see how we can improve things from there.

# 2. Automatic Feature Selection

The preliminary feature selection let us eliminate quite a few features already. However, many still remain. In this section, we'll investigate more ways to eliminate features. 

However, let us first create a baseline model performance with the features that remained after the preliminary feature selection.

In [ ]:
y = food_df_prepared.dropna()['calories']
X = food_df_prepared.dropna().drop(['calories','title'], axis = 'columns')

X.shape, y.shape

The data set is split into feature matrix **X** and target vector **y**.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=111)

**X** and **y** have further been split into a train and a test set. 

We are now ready to create our baseline model.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor()

pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('model', model)])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

We create a modeling  pipeline consisting of a scaler and a model. The k-Nearest Neighbors algorithm calculates the distance between samples based on each feature equally, and is therefore notoriously sensitive to irrelevant features or features with a low predictive value. 

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

# Calculate scores. 
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Report. 
print(f'r2 score: {r2:.3f}')
print(f'MAE: {mae:.3f}')

We choose two metrics to report the effectiveness of our model: $r^2$ and mean absolute error. The $r^2$ score is measured on a 0.0-1.0 scale, with 1.0 being the highest possible value. For the mean average error, the lower the better. 

### 2.1 Statistical testing

Our first approach to further eliminate irrelevant or the least contributing features is through statistical testing. 

`f_regression` is a univariate linear regresson test, that tests for the individual effect of each of the features. The correlation between each feature and the target is computed, which is then converted to an F score and p-value. We can use the p-value to determine which features to eliminate. 

In [ ]:
from sklearn.feature_selection import f_regression

# Perform test. 
_, p_values = f_regression(X_train, y_train)

# Columns to drop. 
columns_to_drop = X_train.columns[p_values > 0.05]

# Convert data into Pandas dataframe. 
X_train_reduced = X_train.drop(columns_to_drop, axis='columns')
X_test_reduced = X_test.drop(columns_to_drop, axis='columns')

print(f'Columns dropped: {columns_to_drop.tolist()}\n')
X_train_reduced.shape, X_test_reduced.shape

14 features were dropped to create the new dataset. Let us see what the effect on our model performance is with this reduced model.

In [ ]:
model = KNeighborsRegressor()

pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('model', model)])

pipeline.fit(X_train_reduced, y_train);
y_pred = pipeline.predict(X_test_reduced)

In [ ]:
# Calculate scores. 
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Report. 
print(f'r2 score: {r2:.3f}')
print(f'MAE: {mae:.3f}')

Removing these features lead to an increase in perfomance, compared to using the original dataset!

A disadvantage, however, is that we still manually needed to create the new version of our dataset. Just like any other data preprocessing technique, we would prefer to transform our data as part of our pipeline. 

An option for that uses `f_regression` is to use `SelectKBest` in our pipeline. This allows us to determine exactly the number of features we want and decide which features to use based on some metric. `SelectKBest` selects features according to the k highest scores. This selector takes two arguments; the function to score the feature importance (typically `f_classif` for classification, `f_regression` for regression) and the number of features to select k (default 10). 

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

model = KNeighborsRegressor()

pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('feature_selection', SelectKBest(f_regression, k=10)),
    ('model', model)])

pipeline.fit(X_train, y_train);
y_pred = pipeline.predict(X_test)

In [ ]:
# Calculate scores. 
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Report. 
print(f'r2 score: {r2:.3f}')
print(f'MAE: {mae:.3f}')

This seems to improve our model performance quite a bit! 

### 2.2 Model-based feature selection

Even with these techniques, selecting the right features can be a tricky problem. This raises an interesting question: shouldn't our models be able to answer this question for us? 

In the k-Nearest Neighbor algorithm, data points are compared to one another by some distance metric, where each feature contributes equally to the calculated distance. This means that irrelevant features can distort our results. 

However, other families of algorithms are less sensitive to redundant or irrelevant features. One such example is Linear Regression.

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('model', model)])

pipeline.fit(X_train, y_train);
y_pred = pipeline.predict(X_test)

In [ ]:
# Calculate scores. 
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Report. 
print(f'r2 score: {r2:.3f}')
print(f'MAE: {mae:.3f}')

The overall performance is better than our k-nearest neighbors model, even with reduced features through `f_regression`. 

The scikit-learn implementation of LinearRegression allows us to investigate the estimated _coefficients_ for the linear regression problem. This should give us an indication of which features contribute most and least to the prediction.

In [ ]:
coef = pipeline['model'].coef_

fig, ax = plt.subplots(figsize = (17,6))
sns.boxplot(data = np.absolute(coef), ax = ax, orient='h');

Looks like most of the (absolute values of) coefficients are either almost zero or very close to it, while a few features with much larger coefficients would be considered outliers. This could be a clear indication that our model is actually mostly using the information contained in a few most informative features.

In [ ]:
# Drop columns with small coefficients.
columns_to_drop = X_train.columns[np.absolute(coef) < 15]
X_train_reduced = X_train.drop(columns_to_drop, axis = 'columns')
X_test_reduced = X_test.drop(columns_to_drop, axis = 'columns')

print(f'Columns dropped: {columns_to_drop.tolist()}\n')
print(f'Features original: {X_train.shape[1]},  features eliminated: {len(columns_to_drop)}')
X_train_reduced.shape, X_test_reduced.shape

In [ ]:
model = LinearRegression()

pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('model', model)])

pipeline.fit(X_train_reduced, y_train);
y_pred = pipeline.predict(X_test_reduced)

In [ ]:
# Calculate scores. 
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Report. 
print(f'r2 score: {r2:.3f}')
print(f'MAE: {mae:.3f}')

Even though we have eliminated a substantial number of features, the performance barely changes!

This approach - using a linear regression model for feature selection and then training a linear regression model on the reduced number of features - does not offer much in terms of increased performance. This does mean that we could have focused on collecting only the remaining features in the future and still have a well performing model. In practice collection of unnecessary features usually means unnecessary costs and less model interpretability. The main advantage of feature selection in this case would be to reduce the data collection requirements.

However, this has also shown that a trained model is succesfully able to identify less relevant features. This opens up the option to perform feature selection through training a linear regression model, while using another algorithm type for the eventual model - for instance, k-nearest neighbors! 

What *scikit-learn* allows us to do in such cases is to introduce feature selection as a new preprocessing step of the pipeline. A tool called `SelectFromModel` allows a preliminary use of a model to determine the most contributing features that should be selected. Then only the selected features are passed further down the pipeline.

In [ ]:
from sklearn.feature_selection import SelectFromModel

model = KNeighborsRegressor()

pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('feature_selection', SelectFromModel(LinearRegression())),
    ('model', model)])

pipeline.fit(X_train, y_train);
y_pred = pipeline.predict(X_test)

In [ ]:
# Calculate scores. 
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Report. 
print(f'r2 score: {r2:.3f}')
print(f'MAE: {mae:.3f}')

Although not as effective as linear regression on its own, a much better performance than the k-nearest neighbors baseline example, and even an improvement on `SelectKBest`! 

In practice, `SelectFromModel` is most often used in combination with what we call a _penalized_ model. Linear regression, for example, has a penalized version called `Lasso` or `L1` regression. By adding a special term to our regular cost function, the model is discouraged from unnecessary model complexity: 

$$ J_{L1}(\mathbf{w})= \frac{1}{n}\sum_{i=1}^n\left(y_i-y( \mathbf{w},\mathbf{x}^i) \right)^2 + \lambda \sum_{j=1}^m \left|w_j\right| $$

`Lasso` regression would punish your linear model for having too many large coefficients for variables that barely contribute for predictions. It will naturally push those coefficients towards zero, and therefore we can use it more easily as a feature selection technique.

For this reason, the two most popular selection models used with `SelectFromModel` are  `Lasso` for regression problems and for classification `LinearSVC(penalty="l1")` - linear support vector machine classifier with an additional Lasso-like penalty term.

### 2.3 Using cross-validation for feature selection


Alternatively, you could also remove features step-wise based on cross-validated performance using `sklearn.feature_selection.RFECV`. This approach can be computationally prohibitive, but some algorithms allow to re-use computations across runs, which leads to a very significant speed boost. 

## Summary

Feature selection is important for many reasons:

- Most real world datasets are very large and hard to comprehend
- Most prediction problems can be solved with only a fraction of features
- Stopping to collect irrelevant features means saved costs
- Many ML models perform much better after feature selection, for some it is almost a must
- Feature selection makes simplifies interpretability of ML models


In this notebook we have considered a variety of approaches for feature selection. Some approaches are immediately available for raw data, where features can be manually excluded if we find:

- Too many **missing values**  
- Too little value **variance** (see [VarianceThreshold](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html#sklearn.feature_selection.VarianceThreshold))
- **High correlation** with the target or other features

Other approaches that can be directly included in the pipeline are: 
- **Statistical testing** where little contributing features are removed (see [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest)). 
- **Model-based selection** where a(n often penalized) model is used to the determine and eliminate the least contributing features (see [SelectFromModel](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel)).



In practice it is advised to try various feature selection techniques and try to be conservative: by trying to remove 'irrelevant' features carelessly, we may accidentally remove integral parts of the data that could have substantially helped the model.